In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool

In [2]:
local_llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

cloud_llm = LLM(
    model="gemini-2.5-flash",
    api_key=os.getenv("GEMINI_API_KEY")
)

In [3]:
planner = Agent(
    role="Content Planner",
    goal="PLan engaging and factually accuratte content on {topic}",
    backstory="""
    You are working on planning a blog article about the topic: {topic}.
    You collect informaiton that helps the audience learn something and make informated decisions.
    Your work is thebasis for the Content Writer to write an article on this topic.
    """,
    verbose=True,
    llm=cloud_llm,
    tools=[SerperDevTool()],
    allow_delegation=False # this agent is not allowed to delegate work to any other agents
)

writer = Agent(
    role="Content Writer",
    goal="Write engaging, insightful and factually accuratte opinion piece about the topic:{topic}",
    backstory="""
    You are working on writing a writing a new opinion piece about the topic: {topic}.
    You base your writing on the work of the Content Planner, who provides an outline and relevant context about the topic.
    You follow the main objectives and direction of the outline, as provided by the Content Planner.
    You also provide objective and impartial insights and back them up with information provide by the Content Planner.
    You acknowledge in your opinion piece when your statements are opinions as opposed to objective statements. 
    """,
    verbose=True,
    llm=cloud_llm,
    tools=[SerperDevTool()],
    allow_delegation=False # this agent is not allowed to delegate work to any other agents
)   
    
editor = Agent(
    role="Editor",
    goal="Edit the given blog post to align with the writing style of the organization.",
    backstory="""
    You are an editor who receives a blog post from the Content Writer.
    Your goal is to reiew the blog post to ensure that it folws journalistic best practives, provides balanced viewpoints
    when providing opinions or asswertions, and also avoids major controversial topics or opinions when possible
    """,
    verbose=True,
    llm=cloud_llm,
    allow_delegation=False # this agent is not allowed to delegate work to any other agents
)


In [4]:
plan = Task(
    description="""
    1. Prioritize the latest trends, key players, and noteworthy news on {topic}.\n
    2. Identify the target audience, considering thier interests and pain points. \n
    3. Develop a detailed content outline including an introduction, key points, and call to action. \n
    4. Include SEO keywords and relevant data or sources.
    """,   
    expected_output="""
    A comprehensive content plan document with an outline, audience analysis, SEO keywords, 
    and resources.
    """,
    agent=planner
)

write = Task(
    description="""
    1. Use the content plan to craft a compelling blog post on {topic}.\n
    2. Incorporate SEO keywords naturally.\n
    3. Sections/ Subtitles are propertly named in an engaging manner.\n
    4. Ensure the post is structured with an engageing introduction, insightful body, and a summarizing conclusion.\n
    5. Proofread for grammatical errors and 'alignment with the brand's voice'.\n
    """,
    expected_output="""
    A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.
    """,
    agent=writer
)

edit = Task(
    description="""
    Proofread the given blog post for grammatical errors and alignment with the brand's voice.
    """,
    expected_output="""
    A well-written blog post in markdown format, ready for publication, each section should have 2 or 3 paragraphs.
    """,
    agent=editor
)

In [5]:
crew = Crew(
    agents = [planner, writer, editor],
    tasks = [plan, write, edit],
    verbose=True # allows to see all logs of the execution
)

In [6]:
result = crew.kickoff(inputs={"topic": "FARA-7B Model"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0dd903a9-cd17-46d1-9b0e-89c4734b9009                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│      1. Prioritize the latest trends, key players, and noteworthy news on FARA-7B Model.                        │
│                                                                                                                 │
│      2. Identify the target audience, considering thier interests and pain points.                              │
│                                                                                                                 │
│      3. Develop a detailed content outline including an introduction, key points, and call to action.           │
│                                                                                                                 │
│      4. Include SEO keywords and relevant data or sources.                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Content Planner                                                                                         │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Editor                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # FARA-7B: Microsoft's Compact AI Agent Redefining Computer Automation                                         │
│                                                                                                                 │
│  Imagine an AI that doesn't just talk *about* using a computer, but *actually uses* it – navigating             │
│  interfaces, clicking buttons, and typing just like a human. This isn't science fiction anymore. Microsoft      │
│  Research has introduced FARA-7B, a groundbreaking 7-billion-parameter agentic small language model (SLM)       │
│  built specifically for intuitive computer interaction. This innovative model is poised to democratize          │
│  sophisticated AI automation, showcasing remarkable efficiency and power previously associated with much        │
│  larger models.                                                                                                 │
│                                                                                                                 │
│  FARA-7B represents a significant leap forward in the field of artificial intelligence, bringing advanced       │
│  capabilities to the fingertips of developers and businesses alike. Its release marks a pivotal moment,         │
│  shifting the focus towards specialized, efficient AI models that can achieve state-of-the-art performance for  │
│  specific tasks, challenging the long-held belief that bigger is always better in AI. In this deep dive, we'll  │
│  explore FARA-7B's capabilities, its underlying technology, a myriad of practical applications, and its         │
│  broader impact on the future of AI and human-computer interaction.                                             │
│                                                                                                                 │
│  ## What is FARA-7B? The Brain Behind Automated Computer Use                                                    │
│                                                                                                                 │
│  At its core, FARA-7B is an agentic SLM meticulously engineered for seamless computer interaction. Unlike       │
│  traditional language models that primarily process and generate text, FARA-7B is designed to perceive user     │
│  interfaces through visual input and execute actions within those environments. It's an AI model that can       │
│  "see" a browser screenshot, understand its context, and then intelligently decide on the next best action to   │
│  achieve a given goal.                                                                                          │
│                                                                                                                 │
│  This **efficient AI model** boasts several **key features and capabilities**. It possesses visual perception,  │
│  allowing it to process detailed browser screenshots to comprehend the current state of a web page. Crucially,  │
│  it combines this perception with sophisticated action prediction, enabling it to pinpoint precise coordinates  │
│  for actions such as clicks, typing text into specific fields, or scrolling through content. What makes         │
│  FARA-7B truly stand out is its capacity for autonomous operation, performing sequences of actions to complete  │
│  complex tasks end-to-end. All of this is packed into a

Output()

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9c91cbae-2399-456f-b8c8-14601494fefe                                                                     │
│  Agent: Editor                                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯